In [2]:
import numpy as np
import pandas as pd
import re
import json
import os
from konlpy.tag import Okt
import sklearn
import pickle
from tqdm import tqdm
import time

from konlpy.tag import Okt
from konlpy.tag import Kkma

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
train = pd.read_table("train.txt", sep="|",encoding = "cp949")
test = pd.read_table("test.txt",sep="|",encoding="cp949")

In [ ]:
train = train.fillna("")
test = test.fillna("")

train['total_text'] = train['text_obj'] + " " + train['text_mthd'] + " " + train['text_deal']
train.head()

test['total_text'] = test['text_obj'] + " " + test['text_mthd'] + " " + test['text_deal']
test.head()

In [3]:
kkma = Kkma()

In [5]:
stop_words_1 = ['은','는','이','가', '에게', '으로',"의",'고', '의해', '해', '을','를', '에서', "로", "대상", "및", "등" ]

In [6]:
def preprocessing(text, okt, remove_stopwords = False, stop_words = []):
    # 함수의 인자는 다음과 같다.
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 생성하지 않고 미리 생성후 인자로 받는다.
    # remove_stopword : 불용어를 제거할지 선택 기본값은 False
    # stop_word : 불용어 사전은 사용자가 직접 입력해야함 기본값은 비어있는 리스트
    
    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    #review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", review)
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    # 중복으로 생성된 공백값을 제거합니다.
    text = re.sub('[\s]+', ' ', text)
    
    # 2. okt 객체를 활용해서 형태소 단위로 나눈다. (형태소분석기 다른거 쓸 때 여기 변경하시면 됩니당!)
    word_review = kkma.morphs(text) #, stem=True)
    
    if remove_stopwords:
        
        # 불용어 제거(선택적)
        word_review = [token for token in word_review if not token in stop_words_1]
        
   
    return word_review

In [ ]:
clean_train_text = []
for i in tqdm(range(len(train['total_text']))):
    clean_train_text.append(preprocessing(train['total_text'][i], okt, remove_stopwords = True, stop_words= stop_words_1))

In [ ]:
clean_test_text = []
for i in tqdm(range(len(test['total_text']))):
    clean_test_text.append(preprocessing(test['total_text'][i], okt, remove_stopwords = True, stop_words= stop_words_1))

In [ ]:
# 저장 
with open('train_text_kkma_morphs불1.pickle', 'wb') as f:
    pickle.dump(clean_train_text, f, pickle.HIGHEST_PROTOCOL)
    
with open('test_text_kkma_morphs불1.pickle', 'wb') as f:
    pickle.dump(clean_test_text, f, pickle.HIGHEST_PROTOCOL)